In [ ]:
from urllib.request import urlopen
import xmltodict
import json

In [ ]:
file = urlopen('http://www.senado.cl/wspublico/tramitacion.php?fecha=05/09/2020')
xmldata = file.read()
file.close()

In [ ]:
data = xmltodict.parse(xmldata, force_list=('indicacion', 'informe', 'oficio', 'tramite',
                                           'urgencia', 'votacion'))

In [ ]:
json_data = json.dumps(data, ensure_ascii=False)

In [ ]:
with open("data/data.json", "w") as json_file:
    json_file.write(json_data)

In [ ]:
projects = json.loads(json_data)

In [ ]:
campos = [['autores', 'autor'], ['comparados', 'comparado'], ['materias', 'materia'],
    ['indicaciones', 'indicacion', 'FECHA'], ['informes', 'informe', 'FECHAINFORME'],
        ['oficios', 'oficio', 'FECHA'], ['tramitacion', 'tramite', 'FECHA'],
        ['urgencias', 'urgencia', 'FECHAINGRESO'], ['votaciones', 'votacion', 'FECHA']]

new_json = []
for proyecto in projects['proyectos']['proyecto']:
    json_project = {}
    json_project['descripcion'] = proyecto['descripcion']
    for campo in campos:
        if proyecto[campo[0]] is not None:
            json_project[campo[0]] = proyecto[campo[0]][campo[1]]
            if campo[0] == 'tramitacion':
                arr = []
                for i,tramite in enumerate(json_project[campo[0]]):
                    if i == 0:
                        obj = {}
                        camara = tramite['CAMARATRAMITE']
                        fecha_inicio = tramite['FECHA']
                    if tramite['CAMARATRAMITE'] != camara:
                        fecha_termino = tramite['FECHA']
                        obj['camara'] = camara
                        obj['inicio'] = fecha_inicio
                        obj['termino'] = fecha_termino
                        arr.append(obj)

                        obj = {}
                        camara = tramite['CAMARATRAMITE']
                        fecha_inicio = tramite['FECHA']
                    if i == len(json_project[campo[0]]) - 1:
                        fecha_termino = tramite['FECHA']
                        obj['camara'] = camara
                        obj['inicio'] = fecha_inicio
                        obj['termino'] = fecha_termino
                        arr.append(obj)
                json_project['camaras'] = arr
        else:
            json_project[campo[0]] = None
            if campo[0] == 'tramitacion':
                json_project['camaras'] = None
            
    new_json.append(json_project)

In [ ]:
with open("data/new_data.json", "w") as json_file:
    this_file = json.dumps(new_json, ensure_ascii=False)
    json_file.write(this_file)